# Object Detection 방식 정리

## 1) 슬라이딩 윈도우(Sliding Window)
---
큰 이미지에서 여러 물체를 찾으려면 어떻게 해야 할까?

<img src="./image/sliding_window.png" />
<center><b>[출처: https://www.researchgate.net/figure/Object-detection-by-sliding-window-approach_fig1_266215670]</b></center>

* 전체 이미지를 적당한 크기의 영역으로 나눈 후에, 각각의 영역에 대해 Localization network를 반복 적용해 보는 방식을 생각해 볼 수 있다. 이러한 방식을 __슬라이딩 윈도우(sliding window)__ 라고 한다. 
* 원본 이미지에서 잘라내는 크기를 윈도우 크기로 하여, 동일한 윈도우 사이즈의 영역을 이동시키면서(sliding) 수행해주는 방식이다.


__Sliding window방식을 사용하지 않는 이유__

* Sliding window방식은 매우 많은 갯수의 window 영역에 대해 이미지의 localization을 돌아가면서 하나씩 진행해야하므로 처리해야할 window 갯수만큼 시간이 더 걸린다.
* 또한 물체의 크기가 다양해지면 단일 크기의 window로 이를 커버할 수 없으므로 더욱 처리속도 문제가 심각해진다.

## 2) 컨볼루션(Convolution)
---

<img src="./image/conv.png" />
<center><b>[출처: https://medium.com/datadriveninvestor/evolution-of-object-recognition-algorithms-i-5803c7be0691]</b></center>

* 위 그림에서 14X14 크기의 입력에 대해 convolution을 수행했을 때 최종적으로 얻어지는 1X1 사이즈의 출력을 sliding window 영역의 localization 결과라고 해석한다면, 
* 거꾸로 14X14 크기의 receptive field가 바로 sliding window 영역이 되는 효과가 있다. 
* __병렬적으로 동시에 진행되므로 convolution은 속도면에서 훨씬 효율적__이다.

## 3) 앵커 박스(Anchor box)
---
* __앵커 박스(anchor box)__는 서로 다른 형태의 물체와 겹친 경우에 대응할 수 있다.
* 일반적으로 차는 좌우로 넓고 사람은 위아래로 길쭉하다. 따라서 사람의 형태와 유사한 형태와 차와 유사한 형태의 가상의 박스 두 개를 정의한다.

차와 사람 클래스에 대해서 물체를 감지하기 위해서는 한 개의 그리드 셀에 대한 결과값 벡터가 물체가 있을 확률, 2개의 클래스, 그리고 바운딩 박스 4개로 총 7개의 차원을 가지게 될 것이다. 따라서 입력값이 16x16일때, 이 그림을 2x2로 총 4칸의 그리드로 나누었다고 하면, 결과값의 형태는 7개의 채널을 가져 2x2x7이 된다.

이때 7개의 차원을 한 벌 더 늘려주어 한 개의 물체의 수를 늘려준다. 앵커 박스가 두 개가 된다면 결과값의 형태는 2x2x14가 된다.

<img src="./image/anchorbox.png" />
<center><b>[출처: http://datahacker.rs/deep-learning-anchor-boxes/]</b></center>

위에서 설명한 앵커 박스는 위와 같다. `Anchor box #1`은 사람을 위해 설정한 크기이고 `Anchor box #2`는 차를 위해 설정한 크기이다. $y$의 라벨을 보면 앵커 박스가 2개가 됨에 따라서 output dimension이 두 배가 된 것을 볼 수 있다. 그리고 각각은 정해진 Anchor box에 매칭된 물체를 책임지게 된다.

__바운딩 박스와 앵커 박스라는 두가지 개념 차이__

>* _바운딩 박스 : 네트워크가 predict한 object의 위치가 표현된 박스로서, 네트워크의 출력이다._
>* _앵커 박스 : 네트워크가 detect해야 할 object의 shape에 대한 가정(assumption)으로서, 네트워크의 입력이다._

## 4) NMS(Non-Max Suppression)
---
<img src="./image/nms.png" />
<center><b>[출처: https://www.quora.com/How-does-non-maximum-suppression-work-in-object-detection]</b></center>

우리가 2x2 또는 더 큰 Grid cell에서 물체가 있는지에 대한 결과를 받게되면 매우 많은 물체를 받게 된다. Anchor box를 사용하지 않더라도 2x2격자에 모두 걸친 물체가 있는 경우 하나의 물체에 대해 4개의 Bounding box를 얻게 된다. 이렇게 겹친 여러 개의 박스를 하나로 줄여줄 수 있는 방법 중 하나가 NMS(non-max suppression)이다.

NMS는 겹친 박스들이 있을 경우 가장 확률이 높은 박스를 기준으로 기준이 되는 IoU 이상인 것들을 없앤다. 이때 IoU를 기준으로 없애는 이유는 어느 정도 겹치더라도 다른 물체가 있는 경우가 있을 수 있기 때문이다. 이때 Non-max suppression은 같은 class인 물체를 대상으로 적용하게 된다.

# 5) 정리
---
* Convolutional implementation of Sliding Windows
    - Convolution으로 슬라이딩 윈도우를 대신함으로써 여러 window에서 Object localization을 병렬로 수행할 수 있게 되어 속도 측면의 개선이 있었다.
* Anchor box
    - Anchor box는 겹친 물체가 있을 때, IoU를 기준으로 서로 다른 Anchor에 할당하도록 하여 생긴 영역이 다른 물체가 겹쳤을 때도 물체를 검출할 수 있도록 할 수 있게 되었다.
* Non-max suppression(NMS)
    - Non-max suppression은 딥러닝 모델에서 나온 Object detection 결과들 중 겹친 결과들을 하나로 줄이면서 더 나은 결과를 얻을 수 있게 했다.